<a href="https://colab.research.google.com/github/halen48/NibabelExmplo/blob/main/Abrindo_arquivos_com_nibabel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Dependências
import time

from IPython.display import Image
import datetime
import tarfile
import matplotlib.pyplot as plt


#Params

In [ ]:
work_path = '/drive/MyDrive/dataset/' #@param{type:"string"}
dataset_path = 'MRI/' #@param{type:"string"}
dataset_labels = "datalabels.csv" #@param{type:"string"}


In [ ]:
#@title Preparar o ambiente
import os
from google.colab import drive

drive.mount('/content/drive')

dir = os.getcwd()+work_path

work_path = '.' + work_path

print(os.listdir(dir))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['runs', 'MNIST', 'GAN_1', 'Discriminators', 'Generators', 'epoche_log.csv', 'GAN_trained.pt', 'Mamografia', '29-07-2021', 'logloss.csv', '09-08-2021', 'Duke MRI Breast Cancer', '064_statistics.csv', '128_statistics.csv', 'Cópia MRI', 'MRI']


#Dataset

>> https://decathlon-10.grand-challenge.org/<br>
>>Extract the files to the work_path
```
files = [f for f in os.listdir(work_path+dataset_path) if(f[-4:] == '.tar')]
for f in files:
  print("Extracting %s"%f)
  with tarfile.open(work_path+dataset_path+f) as tar:
    for sf in tar:
      ''' skip if already extracted '''
      if(sf.isfile() and not os.path.exists(work_path+dataset_path+sf.name) and sf.name.split('/')[-1][:2] != '._'):
        print("Extracting:",sf.name)
        tar.extract(sf, path = work_path+dataset_path)
      else:
        print("Skipping",sf.name)
```

>>Dependencies

In [ ]:
import nibabel
import math
import random
from PIL import Image as imgPIL
import numpy as np
import pandas as pd

>>Dataset criado seguindo <a href="https://pytorch.org/tutorials/beginner/data_loading_tutorial.html">essas</a> instruções

In [ ]:
class MRIDataset():

  def __init__(self, root_dir, csv_file):

    self.landmarks_frame = pd.read_csv(csv_file)
    self.root_dir = root_dir

  def __len__(self):
    return len(self.landmarks_frame)

  def __getitem__(self, idx):
    
    img_name = os.path.join(self.root_dir,
                            self.landmarks_frame.iloc[idx, 0])
    image = nibabel.load(img_name).get_fdata()
    
    landmarks = self.landmarks_frame.iloc[idx, 1]
    landmarks = np.array(landmarks)
    #landmarks = landmarks.astype('float').reshape(-1, 2)
    sample = [np.array(image), img_name, None, landmarks]
    sample[2] = sample[0].shape
    
    return sample

>>Check if the csv file with labels exisits

In [ ]:
roots = [f for f in os.listdir(dir+dataset_path) if os.path.isdir(dir+dataset_path+f)]
#random classes
number_classes = len(roots)
classes = ["Class %d"%i for i in range(0,number_classes)]
#==========================================
try:
  open(dir+dataset_path+dataset_labels)
except:
  print("Generating [%s]"%dataset_labels)
  import pandas as pd
  dataset_folders = [(i,root+'/imagesTr/') for i,root in enumerate(roots)]
  trainin_folders = [(i,root+'/imagesTs/') for i,root in enumerate(roots)]
  extension_img = ".nii.gz"

  files = []
  labels = []
  dims = []

  for i,dataset_folder in (dataset_folders+trainin_folders):
    dim_min = [999,999,999]
    dim_max = [0,0,0]
    for f in os.listdir(dir+dataset_path+dataset_folder):
      if(f[:2] != '._' and f[-len(extension_img):] == extension_img):
        try:
          dims.append(nibabel.load(work_path+dataset_path+dataset_folder+f).get_fdata().shape)
          files.append(dataset_folder+f)
          labels.append(i)
          for l in range(0,3):
            dim_max[l] = max(dims[-1][l], dim_max[l])
            dim_min[l] = min(dims[-1][l], dim_min[l])
          print(files[-1], labels[-1], dims[-1])
        except:
          print("Skipping: %s"%f)
          pass
    print(">>> %s: %s ~ %s"%(dataset_folder,dim_min,dim_max))

  pd.DataFrame({'data':files,'label':labels,'dimensions':dims}).to_csv(dir+dataset_path+dataset_labels, index=False)  

><a href="https://www.youtube.com/watch?v=Tc9ONZLBHP0">Visualizando o dataset</a>

In [ ]:
try:
  os.mkdir('samples')
except FileExistsError:
  pass

dataset = MRIDataset(dir+dataset_path, dir+dataset_path+dataset_labels)

#intensity of image (0~255)
intensity = 255

for i in range(len(dataset)):
  img_data = dataset[i][0]
  print(img_data.shape,dataset[i][1:])
  max_v = np.amax(img_data)
  img_data = intensity*img_data/max_v

  gif = imgPIL.fromarray(img_data[:,:,0])

  frames = [imgPIL.fromarray(img_data[:,:,p]) for p in range(1,img_data.shape[2]) ]

  gif.save('samples/%03d.gif'%i, save_all=True, append_images=frames, duration=1/len(frames), loop = 0)


file_name = 'samples/%03d.gif'%(random.randint(0,batch_size-1))
print("Mostrando %s"%file_name)
Image(open(file_name,'rb').read())
